In [120]:
import pandas as pd
import numpy as np
import re

In [121]:
partialResults = pd.read_csv('./telegram/results.csv')

Despejamos los juegos anteriores al 01/01/2023

In [122]:
partialResults= partialResults[partialResults['gamedate']>'2023-01-01']

In [123]:
uniqueUsers = partialResults['whoid'].unique()
uniqueGames = partialResults['gamenumber'].unique()
columns = ['gamenumber']
  
finalResults = pd.DataFrame(columns=columns)
  #fnaalResults = finalResults.set_index('gamenumber')

In [124]:
partialResults

,gamedate,whoname,whoid,gamenumber,performance
272,2023-01-01T10:25:33,Daniel Modini,user286600771,360,5
273,2023-01-01T10:28:16,Ariel Mantovani,user29274611,360,5
274,2023-01-02T08:36:24,Walter,user75868618,361,6
275,2023-01-02T08:40:57,Daniel Modini,user286600771,361,6
276,2023-01-02T10:14:52,Ariel Mantovani,user29274611,361,4
...,...,...,...,...,...
1399,2023-12-21T23:21:50,Nicolas Lopez barrios,user167977758,714,2
1400,2023-12-22T07:46:18,Daniel Modini,user286600771,715,4
1401,2023-12-22T08:26:14,Ariel Mantovani,user29274611,715,4
1402,2023-12-22T08:29:36,Nicolas Lopez barrios,user167977758,715,4


In [125]:
finalResults['gamenumber']=uniqueGames

In [126]:
finalResults['gamenumber'].describe()


count    351.000000
mean     537.387464
std      102.879670
min      360.000000
25%      449.500000
50%      537.000000
75%      625.500000
max      715.000000
Name: gamenumber, dtype: float64

Aca selecciona los datos por usuario, luego los mergea y renombra la columna.

In [127]:
for user in uniqueUsers:
    userData = partialResults.loc[partialResults['whoid']==user, ('gamenumber','performance')]
    finalResults = pd.merge(finalResults, userData, on='gamenumber', how='left', suffixes=('', '_new'))
    
    userName = partialResults.loc[partialResults['whoid']==user, ('whoname')].unique()
    playername=userName[0]
    if playername.find(' ')>0:
        spaceLength=playername.find(' ')
        playername=playername[:spaceLength]
    finalResults= finalResults.rename(columns={'performance':playername})


In [128]:
finalResults

,gamenumber,Daniel,Ariel,Walter,Nicolas,Gustavo,Roger
0,360,5.0,5.0,NaN,NaN,NaN,NaN
1,361,6.0,4.0,6.0,NaN,NaN,NaN
2,362,3.0,3.0,3.0,NaN,NaN,NaN
3,363,3.0,NaN,NaN,NaN,NaN,NaN
4,364,6.0,4.0,6.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
356,711,NaN,3.0,4.0,3.0,NaN,NaN
357,713,3.0,3.0,3.0,NaN,NaN,NaN
358,712,NaN,7.0,NaN,NaN,NaN,NaN
359,714,5.0,4.0,4.0,2.0,NaN,NaN


Aca obtenemos los usuarios participantes

In [129]:
uniqueUserCols = finalResults.columns
excludedColumn='gamenumber'
mask = [excludedColumn not in x for x in uniqueUserCols]  
userNames = uniqueUserCols[ np.where( mask * uniqueUserCols != '') ]

In [130]:
userNames

Index(['Daniel', 'Ariel', 'Walter', 'Nicolas', 'Gustavo', 'Roger'], dtype='object')

* Analiza la estadistica de Juego de Cada jugador.

In [131]:
finalStats=pd.DataFrame()

In [132]:
for user in userNames:
    userData = finalResults[user].describe()
    finalStats[user] = userData
    

In [133]:
finalStats

,Daniel,Ariel,Walter,Nicolas,Gustavo,Roger
count,322.000000,346.000000,293.000000,200.000000,1.0,2.000000
mean,4.319876,3.806358,4.191126,3.840000,2.0,3.500000
std,1.238093,1.018703,1.068376,1.229618,NaN,0.707107
min,2.000000,2.000000,2.000000,2.000000,2.0,3.000000
25%,3.000000,3.000000,4.000000,3.000000,2.0,3.250000
50%,4.000000,4.000000,4.000000,4.000000,2.0,3.500000
75%,5.000000,4.000000,5.000000,5.000000,2.0,3.750000
max,7.000000,7.000000,7.000000,7.000000,2.0,4.000000


Ahora reemplazemos los valores no votados por 7 para sacar la mayor puntuacion segun el criterio de la competencia

In [134]:
finalResults.fillna(7, inplace=True)

In [135]:
finalResults

,gamenumber,Daniel,Ariel,Walter,Nicolas,Gustavo,Roger
0,360,5.0,5.0,7.0,7.0,7.0,7.0
1,361,6.0,4.0,6.0,7.0,7.0,7.0
2,362,3.0,3.0,3.0,7.0,7.0,7.0
3,363,3.0,7.0,7.0,7.0,7.0,7.0
4,364,6.0,4.0,6.0,7.0,7.0,7.0
...,...,...,...,...,...,...,...
356,711,7.0,3.0,4.0,3.0,7.0,7.0
357,713,3.0,3.0,3.0,7.0,7.0,7.0
358,712,7.0,7.0,7.0,7.0,7.0,7.0
359,714,5.0,4.0,4.0,2.0,7.0,7.0


Ahora calculamos los juegos que superaron los 6 intentos o estuvieron ausentes

In [136]:
for column in finalStats.columns:
        missedlShots = len(finalResults[finalResults[column]== 7])
        finalStats.loc['missed',column] = missedlShots


Llego el momento de calcular los valores totales.

Agrega los valores de las sumas de los totales a finalStats

In [137]:
for column in finalStats.columns:
        totalPoints = finalResults[column].sum()
        finalStats.loc['totals',column] = totalPoints


In [138]:
finalStats

,Daniel,Ariel,Walter,Nicolas,Gustavo,Roger
count,322.000000,346.000000,293.000000,200.000000,1.0,2.000000
mean,4.319876,3.806358,4.191126,3.840000,2.0,3.500000
std,1.238093,1.018703,1.068376,1.229618,NaN,0.707107
min,2.000000,2.000000,2.000000,2.000000,2.0,3.000000
25%,3.000000,3.000000,4.000000,3.000000,2.0,3.250000
50%,4.000000,4.000000,4.000000,4.000000,2.0,3.500000
75%,5.000000,4.000000,5.000000,5.000000,2.0,3.750000
max,7.000000,7.000000,7.000000,7.000000,2.0,4.000000
missed,56.000000,19.000000,72.000000,168.000000,360.0,359.000000
totals,1664.000000,1422.000000,1704.000000,1895.000000,2522.0,2520.000000
